<h1>Redes Sociais: APS 2</h1>

<h3>Professor: Marcelo Hashimoto</h3>

<h3>Alunos: Bernardo Capoferri, Nicolas Queiroga e Raphael Lahiry<h3>

<h1>Carregando o Grafo</h1>

In [1]:
# Importando bibliotecas necessárias para a construção de grafos
import netpixi
import graph_tool.topology
from netpixi.integration.gt import *
from regression.integration.gt import *

/usr/local/lib/python3.10/site-packages/seaborn/rcmod.py:4: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  from distutils.version import LooseVersion
<frozen importlib._bootstrap>:914: ImportWarning: VendorImporter.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: VendorImporter.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: VendorImporter.find_spec() not found; falling back to find_module()


<h1>Reduzindo o Grafo a uma Componente e posicionando os vértices baseado em force-directed graph drawing</h1>

## Criando Grafo da maior componente

In [ ]:
g = gt_load('graph_us_roads.net.gz');

In [ ]:
g = gt_clean(g)

m = gt_draw.sfdp_layout(g)
gt_move(g, m)
gt_save(g, 'graph_us_roads_component.net.gz')
r = netpixi.render('graph_us_roads_component.net.gz')

## Criando segunda maior componente

Devido a limitações de renderização foi necessário escolher uma componente menor para poder visualizar os dados calculados.

### Criando uma lista que classifica a qual componente cada vértice pertence

In [3]:
comp, hist = graph_tool.topology.label_components(g, directed=False)

### Selecionando segundo maior componente do grafo para visualização e salvando-o para um arquivo .net.gz

In [4]:
sum_dic = {}
for node in comp.a:
    if node not in sum_dic:
        sum_dic[node] = 1
    else:
        sum_dic[node] += 1

componente = sorted(sum_dic, key=sum_dic.get)[-2]

In [5]:
# Leitura e do arquivo e construção do grafo

graph_us_roads_component_menor = Graph(directed=False) # grafo não-dirigido

with open('./road-usroads.mtx') as file:
    
    # Pula primeiras 14 linhas para chegar nas linhas com os dados do grafo
    for i in range(14):
        next(file)
        
    graph_data = file.readline().split()
    n_nodes = int(graph_data[1])
    n_edges = int(graph_data[2])
    
    
    # Adicionando vétices ao grafo
    for i in range(n_nodes):
        if comp.a[i] == componente:
            graph_us_roads_component_menor.add_vertex(i+1)
    
    # Adicionando arestas ao grafo
    for edges in file:
        edge_data = edges.split()
        if comp.a[int(edge_data[0])-1] == componente and comp.a[int(edge_data[1])-1] == componente:
            graph_us_roads_component_menor.add_edge(int(edge_data[0]), int(edge_data[1]))

print('Número de vértices do grafo:', graph_us_roads_component_menor.num_vertices())
print('Número de arestas do grafo:', graph_us_roads_component_menor.num_edges())

Número de vértices do grafo: 1923
Número de arestas do grafo: 2279


In [6]:
m = gt_draw.sfdp_layout(graph_us_roads_component_menor)
gt_move(graph_us_roads_component_menor, m)
gt_save(graph_us_roads_component_menor, 'graph_us_roads_component_menor.net.gz')

In [6]:
g = gt_load("graph_us_roads_component.net.gz")
g_menor = gt_load("graph_us_roads_component_menor.net.gz")

In [9]:
print('Número de vértices do grafo:', g.num_vertices())
print('Número de arestas do grafo:', g.num_edges())

Número de vértices do grafo: 126146
Número de arestas do grafo: 161950


<h1>Questões</h1>

<h2>a. O que os vértices da rede representam?</h2>

<h1>Carregando o Grafo</h1>

Os vértices da rede representam ruas dos Estados Unidos.

<h2>b. O que as arestas da rede representam?</h2>

As arestas da rede indicam conexões entre ruas dos Estados Unidos.

<h2>c. O que os grupos formados pela tarefa 4 representam?</h2>

Os grupos formados pela tarefa 4 representam uma grande densidade de ruas que se conectam, muito possivelmente configurando um região urbana.

<h1>Métricas</h1>

<h2>Centralidade Degree</h2>

<h3>a. Calculando a Métrica</h3>

In [10]:
dc = gt_total_degree(g)
dc_menor = gt_total_degree(g_menor)

In [11]:
g.add_vp('degree', dc)
g_menor.add_vp('degree', dc_menor)

In [12]:
gt_save(g_menor, 'graph_degree_component_menor.net.gz')
gt_save(g, 'graph_degree_component.net.gz')

In [ ]:
g = gt_load('graph_degree_component.net.gz')

In [19]:
data = gt_data(g).sort_values(by="degree", ascending=True)
data

,label,degree
id,,
532,532,0.000008
42681,42681,0.000008
42750,42750,0.000008
42776,42776,0.000008
42802,42802,0.000008
...,...,...
111891,111891,0.000048
102661,102661,0.000048
42386,42386,0.000048


<h3>b. Chamando o método vertex_scale da netpixi para usar a métrica como referência para os tamanhos dos vértices</h3>

In [34]:
r = netpixi.render('graph_degree_component_menor.net.gz', infinite=True)

In [36]:
r.vertex_scale('degree', 10, 50)

<h3>c. Salvando uma imagem que mostre com clareza essas diferenças de tamanho</h3>

<h3>d.1. A métrica representa algum conceito? Se sim, qual?</h3>

A métrica degree nos fala quantas arestas estão conectadas à um vértice. No nosso caso, o degree de um vértice representaria a quantidade de ruas ligadas a ele.

<h3>d.2. Os maiores vértices (considerando a tarefa 6b) parecem estar em posições interessantes? Quais?</h3>

Sim. Representando as ruas com maiores conexões, os maiores vértices parecem estar localizados ou em clusters de vértices, ou seja, agrupamentos mais densos de ruas, ou em regiões de conexão entre outros clusters.

<h2>Centralidade Betweenness</h2>

<h3>a. Calculando a Métrica</h3>

In [ ]:
from graph_tool import centrality
g = gt_load("graph_us_roads_component.net.gz")
g_menor = gt_load("graph_us_roads_component_menor.net.gz")

bc, _ = centrality.betweenness(g)
bc_menor, _ = centrality.betweenness(g_menor)

In [23]:
g.add_vp('betweenness', bc)
g_menor.add_vp('betweenness', bc_menor)

In [24]:
gt_save(g_menor, 'graph_betweeness_component_menor.net.gz')
gt_save(g, 'graph_betweeness_component.net.gz')

In [3]:
g = gt_load('graph_betweeness_component.net.gz')

In [4]:
data = gt_data(g).sort_values(by="betweenness", ascending=False)
data

,label,betweenness
id,,
77580,77580,0.213117
78746,78746,0.211596
78261,78261,0.206446
77901,77901,0.206056
78026,78026,0.205337
...,...,...
79419,79419,0.000000
79456,79456,0.000000
79763,79763,0.000000


<h3>b. Chamando o método vertex_scale da netpixi para usar a métrica como referência para os tamanhos dos vértices</h3>

In [7]:
r = netpixi.render('graph_betweeness_component_menor.net.gz', infinite=True)

In [8]:
r.vertex_scale('betweenness', 10, 50)

<h3>c. Salvando uma imagem que mostre com clareza essas diferenças de tamanho</h3>

<h3>d.1. A métrica representa algum conceito? Se sim, qual?</h3>

A métrica representa a soma das proporções de geodésicas que passam pelo nosso vértice. Uma geodésica de um vértice X para um vértice Y é o caminho de menor comprimento possível. No caso da nossa rede, um vértice com um betweenness alto representa uma rua de posição central na infraestrutura como avenidas, já que, pela sua centralidade ela fica relativamente próxima de todas as outras ruas, sendo necessário que passem por elas.

<h3>d.2. Os maiores vértices (considerando a tarefa 6b) parecem estar em posições interessantes? Quais?</h3>

Os maiores vértices da visualização do grafo encontram-se nas fronteiras laterais e percorrendo canais centrais no grafo, assemelhando-se às veias do corpo humano se ramificando à outras veias/vasos.

<h2>Centralidade Closeness</h2>

<h3>a. Calculando a Métrica</h3>

In [10]:
from graph_tool import centrality
g = gt_load("graph_us_roads_component.net.gz")
g_menor = gt_load("graph_us_roads_component_menor.net.gz")

cc = centrality.closeness(g)
cc_menor = centrality.closeness(g_menor)

In [11]:
hcc = centrality.closeness(g, harmonic=True)

In [39]:
g.add_vp('closeness', cc)
g_menor.add_vp('closeness', cc_menor)

In [40]:
gt_save(g_menor, 'graph_closeness_component_menor.net.gz')
gt_save(g, 'graph_closeness_component.net.gz')

In [12]:
g = gt_load('graph_closeness_component.net.gz')

In [13]:
data = gt_data(g).sort_values(by="closeness", ascending=False)
data

,label,closeness
id,,
78122,78122,0.006371
78088,78088,0.006367
78108,78108,0.006365
78155,78155,0.006364
77470,77470,0.006362
...,...,...
36458,36458,0.002923
29963,29963,0.002920
35589,35589,0.002915


<h3>b. Chamando o método vertex_scale da netpixi para usar a métrica como referência para os tamanhos dos vértices</h3>

In [14]:
r = netpixi.render('graph_closeness_component_menor.net.gz', infinite=True)

In [15]:
r.vertex_scale('closeness', 10, 50)

<h3>c. Salvando uma imagem que mostre com clareza essas diferenças de tamanho</h3>

<h3>d.1. A métrica representa algum conceito? Se sim, qual?</h3>

A métrica closeness representa a proximidade de um vértice a todos os outros, lembrando que no grafo proxímidade entre vértices não representam necessáriamente distâncias físicas, apenas quantos vértices são necessários ser percorridos. No caso do grafo das ruas estadunidenses ele representa a quantidade de ruas que é necessário atravessar para acessar todas as outras.

<h3>d.2. Os maiores vértices (considerando a tarefa 6b) parecem estar em posições interessantes? Quais?</h3>

Não, devido a a os valores altos serem muito prominentes e muito similares não é possível distinguir posições interessantes. Embora os vértices menores parecem se encontrar em posições mais remotas em apêndices do grafo, possívelemente representando ruas mais isoladas.

<h2>Clustering Coefficient</h2>

<h3>a. Calculando a Métrica</h3>

In [43]:
from graph_tool import clustering
g = gt_load("graph_us_roads_component.net.gz")
g_menor = gt_load("graph_us_roads_component_menor.net.gz")

lc = clustering.local_clustering(g)
lc_menor = clustering.local_clustering(g_menor)

In [44]:
g.add_vp('clustering', lc)
g_menor.add_vp('clustering', lc_menor)

In [45]:
gt_save(g_menor, 'graph_clustering_component_menor.net.gz')
gt_save(g, 'graph_clustering_component.net.gz')

In [ ]:
g = gt_load('graph_clustering_component.net.gz')

In [ ]:
data = gt_data(g).sort_values(by="clustering", ascending=False)
data

<h3>b. Chamando o método vertex_scale da netpixi para usar a métrica como referência para os tamanhos dos vértices</h3>

In [16]:
r = netpixi.render('graph_clustering_component_menor.net.gz', infinite=True)

In [17]:
r.vertex_scale('clustering', 10, 30)

<h3>c. Salvando uma imagem que mostre com clareza essas diferenças de tamanho</h3>

<h3>d.1. A métrica representa algum conceito? Se sim, qual?</h3>

Sim, o conceito de redundância parece se encaixar bem com o clustering coeficient do nosso grafo, já que ele pode representar quantos caminhos diferentes existem para chegar em um mesmo lugar dentro de um conjunto de ruas vizinhas.

<h3>d.2. Os maiores vértices (considerando a tarefa 6b) parecem estar em posições interessantes? Quais?</h3>

Não parecem estar em posições interessantes já que não estão em nenhum centro de aglomeração entre ruas.

<h2>Lacuas Estruturais</h2>

<h3>a. Calculando a Métrica</h3>

In [48]:
g = gt_load("graph_us_roads_component.net.gz")
g_menor = gt_load("graph_us_roads_component_menor.net.gz")

es = gt_effective_size(g)
es_menor = gt_effective_size(g_menor)

In [49]:
g.add_vp('effsize', es)
g_menor.add_vp('effsize', es_menor)

In [50]:
gt_save(g_menor, 'graph_effective_size_component_menor.net.gz')
gt_save(g, 'graph_effective_size_component.net.gz')

In [ ]:
gt_loadt_load('graph_effective_size_component.net.gz')

In [ ]:
data = gt_data(g).sort_values(by="effsize", ascending=False)
data

<h3>b. Chamando o método vertex_scale da netpixi para usar a métrica como referência para os tamanhos dos vértices</h3>

In [18]:
r = netpixi.render('graph_effective_size_component_menor.net.gz', infinite=True)

In [19]:
r.vertex_scale('effsize', 10, 30)

<h3>c. Salvando uma imagem que mostre com clareza essas diferenças de tamanho</h3>

<h3>d.1. A métrica representa algum conceito? Se sim, qual?</h3>

Sim, parecem representar o inverso da redundância, ou seja, quanto mais eficiente é o caminho entre dois vértices, maior a lacuna estrutural. Para o nosso caso, quanto maior a lacuna estrutural de uma rua, menos caminhos existem para acessar esta rua a partir de um vértice vizinho.

<h3>d.2. Os maiores vértices (considerando a tarefa 6b) parecem estar em posições interessantes? Quais?</h3>

Não parecem estar em posições interessantes, já que não observamos nenhum tipo de padrão que nos chamou a atenção.

<h3>Referências e reconhecimento<h3>

@inproceedings{nr,
     title={The Network Data Repository with Interactive Graph Analytics and Visualization},
     author={Ryan A. Rossi and Nesreen K. Ahmed},
     booktitle={AAAI},
     url={https://networkrepository.com},
     year={2015}
}

Link: https://networkrepository.com/road-usroads.php